# 🐱🐶 Cats vs Dogs Classification using CNN + ResNet50

Цей ноутбук містить повний pipeline: 
- Завантаження даних
- Аугментація
- Побудова моделі
- Навчання
- Оцінка
- Fine-tuning ResNet

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns


In [ ]:
batch_size = 32
img_size = (150, 150)
data_dir = "../../data/cats_vs_dogs"  # Змініть при необхідності
train_dir = f"{data_dir}/train"
validation_dir = f"{data_dir}/validation"
test_dir = f"{data_dir}/test"


In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="binary",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True
)

validation_data = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    labels="inferred",
    label_mode="binary",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=False
)

class_names = train_data.class_names


In [ ]:
resize_and_rescale = layers.Rescaling(1./255)
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.map(lambda x, y: (resize_and_rescale(x), y)).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_data = validation_data.map(lambda x, y: (resize_and_rescale(x), y)).cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
model_augmented = keras.Sequential([
    layers.Input(shape=img_size + (3,)),
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model_augmented.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"]
)


In [ ]:
epochs = 10
history = model_augmented.fit(
    train_data,
    validation_data=validation_data,
    epochs=epochs
)


In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend()
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend()
plt.title("Loss")
plt.show()


In [ ]:
test_data = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    label_mode="binary",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

test_data = test_data.map(lambda x, y: (resize_and_rescale(x), y)).prefetch(buffer_size=AUTOTUNE)

y_true = []
y_pred = []

for x_batch, y_batch in test_data:
    preds = model_augmented.predict(x_batch)
    y_true.extend(y_batch.numpy())
    y_pred.extend((preds > 0.5).astype("int32").flatten())

cm = tf.math.confusion_matrix(y_true, y_pred).numpy()

plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='g', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
img_size_resnet = (224, 224)

train_data_224 = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size_resnet,
    batch_size=batch_size,
    label_mode="binary"
).map(lambda x, y: (resize_and_rescale(x), y))

validation_data_224 = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=img_size_resnet,
    batch_size=batch_size,
    label_mode="binary"
).map(lambda x, y: (resize_and_rescale(x), y))

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=img_size_resnet + (3,))
base_model.trainable = False

inputs = Input(shape=img_size_resnet + (3,))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
outputs = Dense(1)(x)

model_finetuned = Model(inputs, outputs)
model_finetuned.class_names = class_names

model_finetuned.compile(
    optimizer=Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()]
)

model_finetuned.fit(train_data_224, validation_data=validation_data_224, epochs=5)
